In [75]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re

In [76]:
df_train = pd.read_excel("../Raw_data/Data_Train.xlsx")
df_test = pd.read_excel("../Raw_data/Data_Test.xlsx")
df_full = df_train.append(df_test)

/home/sidhartha/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [77]:
df_full['Restaurant_name'] = df_full.Restaurant.apply(lambda x: x.split("_")[1])
location_encode = LabelEncoder()
df_full['transformed_location'] = location_encode.fit_transform(df_full.Location)

In [78]:
city = {"FTI College, Law College Road, Pune":"Pune",
 "Sector 3, Marathalli":"Bangalore",
 "Mumbai Central":"Mumbai",
"Sector 1, Noida":"Delhi",
 "Rmz Centennial, I Gate, Whitefield":"Bangalore",
 "Delhi University-GTB Nagar":"Delhi",
"Yerawada, Pune, Maharashtra":"Pune",
 "Delhi Administration Flats, Timarpur":"Delhi",
 "Moulali, Kolkata":"Kolkata",
"Dockyard Road, Mumbai CST Area":"Mumbai",
 "Pune University":"Pune",
 "Gora Bazar, Rajbari, North Dumdum, Kolkata":"Kolkata",
"D-Block, Sector 63, Noida":"Delhi",
 "Sector 14, Noida":"Delhi",
 "Mico Layout, Stage 2, BTM Layout,Bangalore":"Bangalore",
"Laxman Vihar Industrial Area, Sector 3A, Gurgoan":"Gurgoan",
 "Tiretti, Kolkata":"Kolkata",
 "Sandhurst Road, Mumbai CST Area":"Mumbai",
 "MG Road, Pune":"Pune",
 "Hyderabad Public School, Begumpet":"Hyderabad",
"Majestic":"Bangalore",
 "Chandni Chowk, Kolkata":"Kolkata",
 "Delhi High Court, India Gate":"Delhi",
"Chatta Bazaar, Malakpet, Hyderabad":"Hyderabad",
 "Sector 63A,Gurgaon":"Gurgaon",
 "Delhi Cantt.":"Delhi",
"Tejas Nagar Colony, Wadala West, Mumbai":"Mumbai",
 "Babarpur, New Delhi, Delhi":"Delhi",
"Nathan Road, Mangaldas Road, Pune":"Pune",
 "Panjetan Colony, Malakpet, Hyderabad":"Hyderabad",
"Raja Bazar, Kolkata":"Kolkata",
 "Jaya Nagar, Saidabad, Hyderabad":"Hyderabad",
 "Noorkhan Bazaar, Malakpet, Hyderabad":"Hyderabad",
"Musi Nagar, Malakpet, Hyderabad":"Hyderabad",
 "BTM Layout 1, Electronic City":"Bangalore"}


In [79]:
df_full['City'] = df_full.Location.map(city)
city_enc =LabelEncoder()
df_full['City'] = city_enc.fit_transform(df_full.City)
df_full['Count_cuisins'] = df_full.Cuisines.apply(lambda x: len(x.split(",")))

In [80]:
## Creating a list of final cuisins 

cuisines = []
for i in df_full.Cuisines.values:
    try:
        cuisines.append(i.split(","))
    except:
        cuisines.append(i)
cuisines_cleaned = []
for i in cuisines:
    cuisines_cleaned+=i
    
final_cuisins = []
for i in list(set(cuisines_cleaned)):
    text = re.findall(r'\w+',i)
    text = (" ").join(text)
    final_cuisins.append(text)

In [81]:
map_dict = dict(zip(set(final_cuisins),[i for i in range (len(set(final_cuisins)))]))

In [82]:
def cuisine_clear(cuisine):
    global map_dict
    cuisines = []
    try:
        cuisines.append(cuisine.split(","))
    except:
        cuisines.append(cuisine)
    
    cuisines_cleaned = []
    for i in cuisines:
        cuisines_cleaned+=i
    final_cuisins = []
    for i in list(set(cuisines_cleaned)):
        text = re.findall(r'\w+',i)
        text = (" ").join(text)
        for i,k in  map_dict.items():
            if i == text:
                final_cuisins.append(k)
    return sum(final_cuisins)

In [83]:
df_full['Cuisines_enc'] = df_full.Cuisines.apply(cuisine_clear)

In [84]:
def clean_text(text):
    text=text.lower()
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)
    text = re.sub(r'www.[^ ]+', '', text)
    text = re.sub(r'[a-zA-Z0-9]*www[a-zA-Z0-9]*com[a-zA-Z0-9]*', '', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = [token for token in text.split() if len(token) > 2]
    text = ' '.join(text)
    return text


In [85]:
df_full['Cuisines'] = df_full['Cuisines'].apply(clean_text)
df_full.Location = df_full.Location.apply(clean_text)

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 1), lowercase=False)
train_route = tf.fit_transform(df_full['Cuisines'])


In [87]:
train_route = pd.DataFrame(data=train_route.toarray(), columns=tf.get_feature_names())


In [88]:
train_route.head()

,afghan,african,american,andhra,arabian,asian,assamese,awadhi,bakery,bangladeshi,...,sushi,tamil,tea,tex,thai,tibetan,turkish,vietnamese,wraps,yogurt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.578626,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [89]:
col_1 = list(df_full.columns)
col_2 = list(train_route.columns)

col_1.extend(col_2)
df_full = pd.concat([df_full.reset_index(drop=True),train_route], axis=1,ignore_index=True)
df_full.columns= col_1

In [90]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tf = TfidfVectorizer(ngram_range=(1, 1), lowercase=False)
# train_route = tf.fit_transform(df_full['Location'])
# train_route = pd.DataFrame(data=train_route.toarray(), columns=tf.get_feature_names())

# col_1 = list(df_full.columns)
# col_2 = list(train_route.columns)

# col_1.extend(col_2)
# df_full = pd.concat([df_full.reset_index(drop=True),train_route], axis=1,ignore_index=True)
# df_full.columns= col_1

In [91]:
# df_full['Location_length_string'] = df_full.Location.apply(lambda x: len(x.split(" "))).astype(int)
# df_full['Cuisine_string_length'] = df_full.Cuisines.apply(lambda x: len(x.split(" "))).astype(int)



In [92]:
import re
df_full = df_full[df_full['Average_Cost']!='for']
df_full['Average_Cost'] = df_full['Average_Cost'].apply(lambda x: re.findall(r"\d+",x)[0])

In [93]:
df_full['Minimum_Order'] = df_full['Minimum_Order'].apply(lambda x: re.findall(r"\d+",x)[0])

In [94]:
val = ['-','NEW','Opening Soon','Temporarily Closed']
val1 = ['-']
val2 = ['Opening Soon','Temporarily Closed']

df_full.Rating = df_full.Rating.apply(lambda x:np.nan if x in val else float(x) )
def missing_value_rating(df):
    df.Rating = df.Rating.fillna(df.Rating.mean())
    return df

df_full = df_full.groupby("transformed_location").apply(missing_value_rating)
df_full.Rating = df_full.Rating.fillna(df_full.Rating.mean())


def missing_votes(df):
    df.Votes = df.Votes.fillna(df.Votes.mean()) 
    return df

df_full.Votes = df_full.Votes.apply(lambda x:np.nan if x=='-' else float(x) )
# # df_full.Votes = df_full.Votes.fillna(int(df_full.Votes.mean()))

df_full =df_full.groupby("transformed_location").apply(missing_votes)

In [95]:
def missing_reviews(df):
    df.Reviews = df.Reviews.fillna(df.Reviews.mean()) 
    return df
df_full.Reviews = df_full.Reviews.apply(lambda x:np.nan if x=='-' else float(x))
df_full =df_full.groupby("City").apply(missing_votes)
df_full.Reviews = df_full.Reviews.fillna(df_full.Reviews.mean())

In [96]:
df_full = df_full.loc[:,~df_full.columns.duplicated()]

In [97]:
df_full.drop(['Restaurant', 'Location','Cuisines'],inplace=True,axis=1)

In [98]:
df_test = df_full[df_full.Delivery_Time.isnull()]

In [99]:
df_full.dropna(inplace = True)

In [100]:
df_full.Average_Cost = df_full.Average_Cost.astype(int)
df_full.Minimum_Order = df_full.Minimum_Order.astype(int)
df_full.Restaurant_name = df_full.Restaurant_name.astype(int)
df_full.transformed_location = df_full.transformed_location.astype(int)

In [101]:
del_time_encode = LabelEncoder()
del_time_encode.fit(df_full.Delivery_Time)
df_full.Delivery_Time = del_time_encode.transform(df_full.Delivery_Time)

In [102]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [103]:
X = df_full.drop("Delivery_Time",axis = 1)
Y = df_full.Delivery_Time

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=56)

In [48]:
RC = RandomForestClassifier(n_estimators=1000,n_jobs=-1)

In [49]:
RC.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [50]:
y_pred = RC.predict(X_test)

In [51]:
accuracy_score(y_test,y_pred)

0.800811176205498

In [52]:
# del(RC)

In [45]:
from sklearn.model_selection import cross_validate

In [46]:
cross_validate(RC,X,Y,error_score='accuracy',cv= 5)

/home/sidhartha/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


{'fit_time': array([6.36495256, 4.62283087, 4.65142035, 4.60709715, 4.65814662]),
 'score_time': array([0.40925217, 0.3067739 , 0.30709672, 0.30597687, 0.30603623]),
 'test_score': array([0.7954955 , 0.79144144, 0.80396575, 0.78764653, 0.79196751])}

In [35]:
# from sklearn.feature_selection import RFECV

In [36]:
# recv = RFECV(RC,step= 10,min_features_to_select=50,scoring='accuracy',n_jobs=-1)

In [37]:
# recv.fit(X_train,y_train)

In [38]:
# # X_train = X_train.iloc[:,recv.get_support(indices=True)]
# X_test = X_test.iloc[:,recv.get_support(indices=True)]


In [105]:
from sklearn.model_selection import RandomizedSearchCV

In [131]:
# params= {'n_estimators' : [500,700,900,1000,1200],
#          'max_depth':[50,100,200,300,400],
#          'learning_rate': [0.1,0.01,0.05,0.2],
#          'min_child_weight':[0.1,0.01,0.001,1,2],
#          'max_delta_step':[40,50,80,100],
#          'reg_alpha':[1,0.1,0.3,0.01],
#          'reg_lambda':[1,0.1,5],
#          'n_jobs':[2]
#          }

In [132]:
# rscv = RandomizedSearchCV(xgb,scoring = 'accuracy',param_distributions=params,verbose=1,n_iter=30,n_jobs=4)

In [ ]:
# rscv.fit(X,Y)

/home/sidhartha/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/home/sidhartha/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 131.9min


In [106]:
# rscv.best_estimator_.fit(X_train,y_train)

In [107]:
# y_pred=rscv.best_estimator_.predict(X_test)

In [108]:
# accuracy_score(y_test,y_pred)

In [109]:
from xgboost import XGBClassifier

In [110]:
xgb = XGBClassifier(n_estimators=1500,learning_rate=0.01,max_depth=410,n_jobs=-1,min_child_weight=0.01)

In [111]:
xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=410,
              min_child_weight=0.01, missing=None, n_estimators=1500, n_jobs=-1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [113]:
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.8156827399729608

In [76]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(n_estimators=500,num_leaves=50,learning_rate=0.01)
lgbm.fit(X_train,y_train)
y_pred = lgbm.predict(X_test)
accuracy_score(y_test,y_pred)

0.7661108607480848

In [119]:
xgb.fit(X,Y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=410,
              min_child_weight=0.01, missing=None, n_estimators=1500, n_jobs=-1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [120]:
df_test.Average_Cost = df_test.Average_Cost.astype(int)
df_test.Minimum_Order = df_test.Minimum_Order.astype(int)
df_test.Restaurant_name = df_test.Restaurant_name.astype(int)
df_test.transformed_location = df_test.transformed_location.astype(int)

In [121]:
y_pred = xgb.predict(df_test.drop("Delivery_Time",axis=1)[X.columns])
df_submit = pd.DataFrame({"Delivery_Time":del_time_encode.inverse_transform(y_pred)})

In [122]:
df_submit.to_excel("submit.xlsx",index = False)